In [1]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
#from torchvision import Datasets
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import random

In [2]:
data=pd.read_csv('engfrench.csv')

In [3]:
print(len(data))

175621


In [4]:
data.iloc[170000]

English words/sentences    The guards found a hacksaw blade in the prison...
French words/sentences     Les gardiens trouvèrent une lame de scie à mét...
Name: 170000, dtype: object

In [87]:
data[10000:10005]

,English words/sentences,French words/sentences
10000,Be very careful.,Sois très prudente !
10001,Bees make honey.,Les abeilles font du miel.
10002,Behave yourself.,Comporte-toi bien.
10003,Bite the bullet.,Serre les dents.
10004,Bite the bullet.,Serrez les dents.


In [5]:
from transformers import BertTokenizer

In [6]:
tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\maddo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maddo\.cache\huggingface\hub\models--google-bert--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [10]:
PAD_TOKEN_ID=0

In [11]:
tokenizer.add_special_tokens({'additional_special_tokens': [' _ENFR_ ']})

0

In [12]:
SOS_TOKEN_ID = tokenizer.cls_token_id if tokenizer.cls_token_id is not None else 0
EOS_TOKEN_ID = tokenizer.sep_token_id if tokenizer.sep_token_id is not None else 0

In [15]:
txten=data.iloc[0,0]
txtfr=data.iloc[0,1]


In [16]:
txten

'Hi.'

In [17]:
txtfr

'Salut!'

In [18]:
token=tokenizer.tokenize(txten)

In [19]:
token

['hi', '.']

In [20]:
tokenfr=tokenizer.tokenize(txtfr)

In [21]:
tokenfr

['sal', '##ut', '!']

In [22]:
len(token)

2

In [23]:
len(tokenfr)

3

In [24]:

dat=txten+' _ENFR_ '+txtfr

In [25]:
dat

'Hi. _ENFR_ Salut!'

In [26]:
tokdat=(tokenizer.tokenize(dat))


In [27]:
print(tokdat)

['hi', '.', ' _ENFR_ ', 'sal', '##ut', '!']


In [28]:
tokdat=tokenizer.convert_tokens_to_ids(tokdat)

In [29]:
for i in range(len(tokdat)):
    print(tokdat[:i],tokdat[i])

[] 7632
[7632] 1012
[7632, 1012] 30522
[7632, 1012, 30522] 16183
[7632, 1012, 30522, 16183] 4904
[7632, 1012, 30522, 16183, 4904] 999


In [30]:
class dataset(Dataset):
    def __init__(self,x):
        self.x=x
        
    def __len__(self):
        return len(self.x)
    def __getitem__(self,i):
        inp=self.x
        return torch.tensor(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(inp.iloc[i,0]))),torch.tensor(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(inp.iloc[i,1])))
        



In [31]:
datanewdataset=dataset(data)

In [32]:
tokenizer.decode(datanewdataset[2000][1])

"ne t'en va pas!"

In [33]:
tokenizer.decode(datanewdataset[2000][0])

"don't leave!"

In [34]:
def custom_collate_fn(batch):
    # 'batch' is a list of tuples: [(input_tensor_1, target_tensor_1), (input_tensor_2, target_tensor_2), ...]

    # 1. Separate inputs and targets
    inputs = [torch.tensor(item[0]) for item in batch]
    targets = [torch.tensor(item[1]) for item in batch]

    # 2. Pad the inputs and targets to the length of the longest sequence in the batch
    # batch_first=True makes the output shape (BatchSize, SequenceLength)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=PAD_TOKEN_ID)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=PAD_TOKEN_ID)
    
    return inputs_padded, targets_padded

In [35]:
haha=DataLoader(datanewdataset,batch_size=32,shuffle=True,collate_fn=custom_collate_fn)

In [36]:
embedsize=len(tokenizer.get_vocab())

In [37]:
print(embedsize)

30523


In [38]:
VOCAB_SIZE=embedsize+5

In [39]:
for i,x in enumerate(haha):
    print(tokenizer.decode(x[1][0]))
    print(tokenizer.decode(x[0][0]))
    break

vous m'avez prevenu, mais je n'ai pas ecoute. [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
you warned me, but i didn't listen. [PAD] [PAD]


C:\Users\maddo\AppData\Local\Temp\ipykernel_1896\1408414250.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = [torch.tensor(item[0]) for item in batch]
C:\Users\maddo\AppData\Local\Temp\ipykernel_1896\1408414250.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = [torch.tensor(item[1]) for item in batch]


In [40]:
xa=torch.randn(10,100)

In [41]:
ha=nn.LSTM(100,200)

In [42]:
ya=ha(xa)

In [43]:
ya[0].shape

torch.Size([10, 200])

In [44]:
ya[1][0].shape

torch.Size([1, 200])

In [45]:
ya[1][1].shape

torch.Size([1, 200])

In [63]:
embed_dim=512

In [64]:
class enc(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb=nn.Embedding(VOCAB_SIZE,embed_dim)
        self.lstm=nn.LSTM(input_size=embed_dim,hidden_size=1024,num_layers=3,batch_first=True,bidirectional=False)
        self.rel=nn.ReLU()
    def forward(self,x):
        embx=self.emb(x)
        out,(hid,cell)=self.lstm(embx)
        return hid,cell
        

        
        

In [65]:
class dec(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(VOCAB_SIZE, embed_dim)
        self.lstm = nn.LSTM(input_size=embed_dim, hidden_size=1024, num_layers=3, batch_first=True, bidirectional=False)
        self.fc_out = nn.Linear(1024, VOCAB_SIZE) # Map hidden state to Vocab Size
        
    def forward(self, x, hidden, cell):
        # x shape: [batch_size] -> unsqueeze to [batch_size, 1]
        x = x.unsqueeze(1)
        embx = self.emb(x)
        
        output, (hidden, cell) = self.lstm(embx, (hidden, cell))
        
        # prediction shape: [batch_size, VOCAB_SIZE]
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

In [66]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        # source: [batch, src_len]
        # target: [batch, trg_len]
        
        batch_size = source.shape[0]
        target_len = target.shape[1]
        vocab_size = VOCAB_SIZE
        
        outputs = torch.zeros(batch_size, target_len, vocab_size).to(self.device)
        
        # 1. Encode
        hidden, cell = self.encoder(source)
        
        # 2. First input to decoder (Start Token)
        # We use the SOS_TOKEN_ID defined at the top
        input_token = torch.full((batch_size,), SOS_TOKEN_ID, dtype=torch.long).to(self.device)
        
        # 3. Decode
        for t in range(target_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, t, :] = output
            
            # Teacher forcing
            top1 = output.argmax(1)
            use_teacher = random.random() < teacher_forcing_ratio
            
            # If using teacher forcing, next input is actual target token
            # Else, next input is predicted token
            if t < target_len - 1:
                input_token = target[:, t] if use_teacher else top1
            
        return outputs

In [67]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

encoder = enc()
decoder = dec()
model = Seq2Seq(encoder, decoder, device).to(device)

# Optimization
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN_ID)

Using device: cuda


In [90]:
def train_one_epoch(model, iterator, optimizer, criterion,epoch, clip=1):
    model.train()
    epoch_loss = 0
    for kek in range(epoch):
        for i, (src, trg) in enumerate(iterator):
            src = src.to(device)
            trg = trg.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            output = model(src, trg)
            
            # Reshape for Loss
            # output: [batch, len, vocab] -> [batch*len, vocab]
            # trg: [batch, len] -> [batch*len]
            output_dim = output.shape[-1]
            output = output.view(-1, output_dim)
            trg = trg.view(-1)
            
            loss = criterion(output, trg)
            loss.backward()
            
            # Clip gradients to prevent explosion
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()
            
            epoch_loss += loss.item()
            
            if i % 200 == 0:
                print(f"Batch {i}, Loss: {loss.item()}")
                
        return epoch_loss / len(iterator)

In [91]:
def predict_translation(model, sentence, max_len=20):
    model.eval()
    
    # Tokenize input
    tokens = tokenizer.tokenize(sentence)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    src_tensor = torch.tensor(ids, dtype=torch.long).unsqueeze(0).to(device) # [1, seq_len]
    
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
    
    # Start with SOS token
    inputs = [SOS_TOKEN_ID]
    
    for _ in range(max_len):
        input_tensor = torch.tensor([inputs[-1]], dtype=torch.long).to(device)
        
        with torch.no_grad():
            output, hidden, cell = model.decoder(input_tensor, hidden, cell)
            
        pred_token = output.argmax(1).item()
        inputs.append(pred_token)
        
        # Stop if EOS or padding is predicted
        if pred_token == EOS_TOKEN_ID or pred_token == PAD_TOKEN_ID:
            break
            
    # Convert IDs back to tokens
    predicted_tokens = tokenizer.convert_ids_to_tokens(inputs[1:]) # Skip SOS
    return tokenizer.convert_tokens_to_string(predicted_tokens)

In [78]:
# Example run (Uncomment to train)
print("Starting Training...")
loss = train_one_epoch(model, haha, optimizer, criterion,5)
print(f"Epoch Loss: {loss}")

Starting Training...
Batch 0, Loss: 4.165121555328369


C:\Users\maddo\AppData\Local\Temp\ipykernel_1896\1408414250.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = [torch.tensor(item[0]) for item in batch]
C:\Users\maddo\AppData\Local\Temp\ipykernel_1896\1408414250.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = [torch.tensor(item[1]) for item in batch]


Batch 200, Loss: 4.199960708618164
Batch 400, Loss: 3.232105255126953
Batch 600, Loss: 3.4725232124328613
Batch 800, Loss: 3.3705008029937744
Batch 1000, Loss: 3.242098093032837
Batch 1200, Loss: 3.515153169631958
Batch 1400, Loss: 3.595393180847168
Batch 1600, Loss: 4.001532077789307
Batch 1800, Loss: 2.679044008255005
Batch 2000, Loss: 2.5007336139678955
Batch 2200, Loss: 3.3220736980438232
Batch 2400, Loss: 2.552337884902954
Batch 2600, Loss: 2.869300603866577
Batch 2800, Loss: 2.706411361694336
Batch 3000, Loss: 2.758835554122925
Batch 3200, Loss: 2.8631083965301514
Batch 3400, Loss: 2.3348615169525146
Batch 3600, Loss: 2.8709332942962646
Batch 3800, Loss: 2.9423513412475586
Batch 4000, Loss: 2.77290678024292
Batch 4200, Loss: 2.5807883739471436
Batch 4400, Loss: 2.520583152770996
Batch 4600, Loss: 2.343597888946533
Batch 4800, Loss: 2.544912099838257
Batch 5000, Loss: 2.928825855255127
Batch 5200, Loss: 2.6749987602233887
Batch 5400, Loss: 2.659404754638672
Epoch Loss: 2.980877454

In [81]:
translation = predict_translation(model, "I am doing great , WHat about you?")
print(f"Translation: {translation}")

Translation: je fais ce cette , , , n ' est - ce pas ? ? ? - vo


In [82]:
translation = predict_translation(model, "Today's a good day.")
print(f"Translation: {translation}")

Translation: aujourd ' hui est un bon jour . . . . . . . . . .


In [84]:
translation = predict_translation(model, "How was prison ?")
print(f"Translation: {translation}")

Translation: comment etait la ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?


In [89]:
translation = predict_translation(model, "Bite the bullet.")
print(f"Translation: {translation}")

Translation: les la la ville . . . . . . . . . . . . . . . .
